In [1]:
# Install essential packages
!pip install rdflib transformers torch datasets sentencepiece sparqlwrapper accelerate --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.7 MB/s eta 0:00:00


In [2]:
from rdflib import Graph
from transformers import T5ForConditionalGeneration, T5Tokenizer, Trainer, TrainingArguments
from datasets import Dataset
import torch
import pandas as pd
import os


In [3]:
from google.colab import files
uploaded = files.upload()

ttl_file = list(uploaded.keys())[0]
print(f"📁 Loaded: {ttl_file}")


Saving Polyhouse Ontology.ttl to Polyhouse Ontology.ttl
📁 Loaded: Polyhouse Ontology.ttl


In [4]:
def extract_triples(ttl_path):
    g = Graph()
    g.parse(ttl_path, format="turtle")
    triples = []
    for s, p, o in g:
        triples.append((str(s), str(p), str(o)))
    return triples

triples = extract_triples(ttl_file)
print(f"✅ Extracted {len(triples)} RDF triples")
triples[:5]  # Sample


✅ Extracted 305 RDF triples


[('http://www.ontology-of-units-of-measure.org/resource/om-2/Temperature',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://www.w3.org/2002/07/owl#Class'),
 ('http://www.ontology-of-units-of-measure.org/resource/om-2/RelativeHumidity',
  'http://www.w3.org/1999/02/22-rdf-syntax-ns#type',
  'http://www.w3.org/2002/07/owl#Class'),
 ('http://www.w3.org/ns/sosa/Observation',
  'http://www.w3.org/2000/01/rdf-schema#subClassOf',
  'http://amrita.sony.org/terms/PolyhouseOntology#SSN'),
 ('http://www.ontology-of-units-of-measure.org/resource/om-2/Temperature',
  'http://www.w3.org/2000/01/rdf-schema#subClassOf',
  'http://www.ontology-of-units-of-measure.org/resource/om-2/Quantity'),
 ('https://pi.pauwel.be/voc/distributionelement#Controller',
  'http://www.w3.org/2000/01/rdf-schema#subClassOf',
  'https://pi.pauwel.be/voc/distributionelement#DistributionControlElement')]

In [5]:
def generate_synthetic_qa(triples):
    examples = []
    for subj, pred, obj in triples:
        question = f"What is the {pred.split('/')[-1]} of {subj.split('/')[-1]}?"
        sparql = f"SELECT ?x WHERE {{ <{subj}> <{pred}> ?x . }}"
        examples.append({"input": question, "output": sparql})
    return examples

examples = generate_synthetic_qa(triples)
df = pd.DataFrame(examples)
df.head()


,input,output
0,What is the 22-rdf-syntax-ns#type of Temperature?,SELECT ?x WHERE { <http://www.ontology-of-unit...
1,What is the 22-rdf-syntax-ns#type of RelativeH...,SELECT ?x WHERE { <http://www.ontology-of-unit...
2,What is the rdf-schema#subClassOf of Observation?,SELECT ?x WHERE { <http://www.w3.org/ns/sosa/O...
3,What is the rdf-schema#subClassOf of Temperature?,SELECT ?x WHERE { <http://www.ontology-of-unit...
4,What is the rdf-schema#subClassOf of distribut...,SELECT ?x WHERE { <https://pi.pauwel.be/voc/di...


In [6]:
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)

def tokenize(batch):
    input_enc = tokenizer(batch["input"], padding="max_length", truncation=True, max_length=128)
    target_enc = tokenizer(batch["output"], padding="max_length", truncation=True, max_length=128)
    return {
        "input_ids": input_enc["input_ids"],
        "attention_mask": input_enc["attention_mask"],
        "labels": target_enc["input_ids"]
    }

dataset = Dataset.from_pandas(df)
tokenized_dataset = dataset.map(tokenize, batched=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/305 [00:00<?, ? examples/s]

In [7]:
model = T5ForConditionalGeneration.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir="./sparql_t5",
    per_device_train_batch_size=4,
    num_train_epochs=5,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    remove_unused_columns=False,
    fp16=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: kanish-prabakaran04 (kanish-prabakaran04-amrita-vishwa-vidyapeetham) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
10,6.006400
20,3.272000
30,2.384800
40,2.017600
50,1.662500
60,1.441100
70,1.216300
80,1.027900
90,0.920100
100,0.854200


TrainOutput(global_step=385, training_loss=0.8277444923078859, metrics={'train_runtime': 2123.2121, 'train_samples_per_second': 0.718, 'train_steps_per_second': 0.181, 'total_flos': 51599061811200.0, 'train_loss': 0.8277444923078859, 'epoch': 5.0})

In [8]:
model_path = "./sparql_t5_model"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)
print(f"✅ Model saved to {model_path}")


✅ Model saved to ./sparql_t5_model


In [21]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load model
model = T5ForConditionalGeneration.from_pretrained(model_path)
tokenizer = T5Tokenizer.from_pretrained(model_path)

def generate_sparql(question):
    input_text = question
    inputs = tokenizer.encode(input_text, return_tensors="pt", truncation=True)
    output_ids = model.generate(inputs, max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Test
test_question = "	What is the rdf-schema#subClassOf of Temperature? "
print("🧪 Question:", test_question)
print("🧠 SPARQL:", generate_sparql(test_question))


🧪 Question: 	What is the rdf-schema#subClassOf of Temperature? 
🧠 SPARQL: SELECT?x WHERE  http://www.w3.org/1999/02/Rdf-schema#subClassOf>?x.
